## Poetry Classification

In [ ]:
!pip install tkseem

In [ ]:
!wget https://raw.githubusercontent.com/ARBML/tkseem/master/tasks/meter_classification/meters/data.txt
!wget https://raw.githubusercontent.com/ARBML/tkseem/master/tasks/meter_classification/meters/labels.txt

### Imports

In [1]:
import tensorflow as tf
import tkseem as tk
import numpy as np
from tensorflow.keras.layers import GRU, Embedding, Dense, Input, Dropout, Bidirectional, BatchNormalization, Flatten, Reshape
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

### Tokenization

In [2]:
tokenizer = tk.CharacterTokenizer(clean = True)
tokenizer.process_data('data.txt')
tokenizer.train()

Reading the data ...
Cleaning the data ...
Training CharacterTokenizer ...


### Data

In [3]:
sentences = open('data/raw/train.txt', 'r').read().splitlines()
labels = open('labels.txt', 'r').read().splitlines()

max_length = 100 

X = tokenizer.encode_sentences(sentences, max_length = max_length)
y = np.array([int(lbl) for lbl in labels])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [4]:
sentences[0]

'يا ضيف طيف ما هداه لمضجعي # إلا لهيب في الحشى يتوقد'

### Model

In [9]:
model = Sequential()
model.add(Input((100,)))
model.add(Embedding(tokenizer.vocab_size, 256))
model.add(Bidirectional(GRU(units = 256, return_sequences=True)))
model.add(Bidirectional(GRU(units = 256, return_sequences=True)))
model.add(Bidirectional(GRU(units = 256)))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(14, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [10]:
model.fit(X_train, y_train, validation_split = 0.1, epochs = 10, batch_size= 256, shuffle = True)

Epoch 1/10
111/111 [==============================] - 464s 4s/step - loss: 2.2710 - accuracy: 0.1868 - val_loss: 1.9023 - val_accuracy: 0.2929
Epoch 2/10
111/111 [==============================] - 465s 4s/step - loss: 1.8318 - accuracy: 0.3188 - val_loss: 1.6296 - val_accuracy: 0.3949
Epoch 3/10
111/111 [==============================] - 453s 4s/step - loss: 1.5879 - accuracy: 0.4187 - val_loss: 1.4511 - val_accuracy: 0.4842
Epoch 4/10
111/111 [==============================] - 400s 4s/step - loss: 1.3652 - accuracy: 0.5193 - val_loss: 1.2189 - val_accuracy: 0.5754
Epoch 5/10
111/111 [==============================] - 393s 4s/step - loss: 1.0705 - accuracy: 0.6416 - val_loss: 0.9010 - val_accuracy: 0.6979
Epoch 6/10
111/111 [==============================] - 393s 4s/step - loss: 0.7646 - accuracy: 0.7536 - val_loss: 0.6613 - val_accuracy: 0.7840
Epoch 7/10
111/111 [==============================] - 389s 4s/step - loss: 0.5865 - accuracy: 0.8191 - val_loss: 0.5526 - val_accuracy: 0.8325

### Test

In [11]:
label2name = ['السريع', 'الكامل', 'المتقارب', 'المتدارك', 'المنسرح', 'المديد', 
              'المجتث', 'الرمل', 'البسيط', 'الخفيف', 'الطويل', 'الوافر', 'الهزج', 'الرجز']

In [12]:
def classify(sentence):
    sequence = tokenizer.encode_sentences([sentence], max_length = max_length)
    pred = model.predict(sequence)[0]
    print(label2name[np.argmax(pred, 0).astype('int')], np.max(pred))

In [13]:
classify("ما تردون على هذا المحب # دائبا يشكو إليكم في الكتب")
classify("ولد الهدى فالكائنات ضياء # وفم الزمان تبسم وسناء")
classify(" لك يا منازل في القلوب منازل # أقفرت أنت وهن منك أواهل")
classify("ومن لم يمت بالسيف مات بغيره # تعددت الأسباب والموت واحد")
classify("أنا النبي لا كذب # أنا ابن عبد المطلب")
classify("هذه دراهم اقفرت # أم ربور محتها الدهور")
classify("هزجنا في بواديكم # فأجزلتم عطايانا")
classify("بحر سريع ماله ساحل # مستفعلن مستفعلن فاعلن")
classify("مَا مَضَى فَاتَ وَالْمُؤَمَّلُ غَيْبٌ # وَلَكَ السَّاعَةُ الَّتِيْ أَنْتَ فِيْهَا")
classify("يا ليلُ الصبّ متى غدهُ # أقيامُ الساعة موعدهُ")

الرمل 0.99949336
الكامل 0.9249198
الكامل 0.9218475
الطويل 0.9974923
المجتث 0.6107578
المديد 0.9062244
الهزج 0.9060462
السريع 0.832686
المتدارك 0.6292933
المتدارك 0.60360384
